In [14]:
#!pip install pyModeS

In [15]:
#!pip install jupyter_dash

In [1]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path
import pyModeS as pms
from pyModeS import adsb
import base64
import tarfile
from datetime import datetime
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output, dash_table
import plotly.express as px

C:\Users\nacho\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\_jupyter.py:28: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  _dash_comm = Comm(target_name="dash")


- Este notebook se enfoca en el análisis de mensajes ADS-B de aviones en tierra y en vuelo.
- Se cargan los datos usando Dask para manejar grandes volúmenes de información de manera eficiente.
- Se procesan los datos mediante filtrado, agrupación y cálculo de la posición geográfica de los aviones.
- Se crea una aplicación Dash interactiva que permite al usuario consultar los vuelos según la hora seleccionada.
- El mapa generado muestra las ubicaciones de un radar y varias pistas de aterrizaje.
- Los aviones en tierra se visualizan en amarillo y los aviones en vuelo en verde, facilitando su identificación.

## **Ejercicio 2**

### Procesamiento
Se carga el archivo CSV con Dask, convirtiendo la columna `Timestamps` a formato `datetime` y extrayendo la hora. Luego, se calcula el `type_code` de los mensajes ADS-B y se filtran los registros correspondientes a tipos de mensajes específicos. A continuación, se aplica una función para determinar si el formato CPR de los mensajes es "Odd" o "Even" y se añade esta información como una nueva columna `'CPR format'` a los datos filtrados.


In [2]:
# Ruta al archivo CSV
path = Path("output_by_day/data_2024-12-01.csv")

# Leer el archivo CSV con Dask
df_vuelos = dd.read_csv(path)

# Convertir la columna Timestamps a datetime
df_vuelos["Timestamps"] = dd.to_datetime(df_vuelos["Timestamps"])

# Extraer la hora de la columna Timestamps
df_vuelos["Hora"] = df_vuelos["Timestamps"].dt.hour

df_vuelos['type_code'] = df_vuelos.map_partitions(
    lambda df: df['message'].apply(lambda msg: pms.common.typecode(msg)),
    meta=('message', 'int64') 
)

# Filtrar las filas cuyo 'type_code' esté en el rango [5, 8] o [9, 18]
df_vuelos_filtrado = df_vuelos[df_vuelos['type_code'].between(5, 8, inclusive='both')]

# Función para determinar el formato CPR (Odd o Even)
def determinar_cpr_format(row):
    tc = row['type_code']
    msg = row['message']
    if 5 <= tc <= 8:  # Surface position
        oe = adsb.oe_flag(msg)
        return "Odd" if oe else "Even"
    else:
        return None

# Aplicar la función para añadir la columna 'CPR format'
df_vuelos_filtrado['CPR format'] = df_vuelos_filtrado.map_partitions(
    lambda df: df.apply(determinar_cpr_format, axis=1),
    meta=('CPR format', 'object')
)

In [3]:
# Computar el DataFrame filtrado
df_vuelos = df_vuelos_filtrado.compute()

# Mostrar el resultado
df_vuelos

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format
19,1733011203261,9034234428000631d27c9309c1c2,342344,<NA>,2024-12-01 00:00:03,NaN,NaN,0.0,0,5.0,Odd
29,1733011203261,90342345280006345c7fb948f79b,342345,<NA>,2024-12-01 00:00:03,NaN,NaN,0.0,0,5.0,Odd
32,1733011203261,9034234628000648b87fb0e9b054,342346,<NA>,2024-12-01 00:00:03,NaN,NaN,0.0,0,5.0,Odd
35,1733011203261,9134108f402008015a6eb67c7e75,34108f,<NA>,2024-12-01 00:00:03,0.125,NaN,0.0,0,8.0,Even
37,1733011203261,8c3441152b78e1bce06544f61162,344115,False,2024-12-01 00:00:03,31.000,39.3750,0.0,0,5.0,Even
...,...,...,...,...,...,...,...,...,...,...,...
891743,1733084001652,913440d742bf3e1f3682e1ab4a84,3440d7,<NA>,2024-12-01 20:13:21,19.000,323.4375,0.0,20,8.0,Odd
891763,1733084001652,9034234e28000613a881d8ac523d,34234e,<NA>,2024-12-01 20:13:21,NaN,NaN,0.0,20,5.0,Odd
891777,1733084001652,8c0201773a39f62b927fea331dbe,020177,False,2024-12-01 20:13:21,13.000,87.1875,0.0,20,7.0,Odd
891783,1733084001652,8c34745339de03f67c6b3df19a11,347453,False,2024-12-01 20:13:21,10.000,270.0000,0.0,20,7.0,Even


Se agrupan los datos por `ICAO` y `Hora`, y se define una función para seleccionar pares de mensajes con formatos CPR "Even" y "Odd", asegurando que estén consecutivos y en el orden correcto. Esta función se aplica a cada grupo y el resultado se almacena en un nuevo DataFrame, que se resetea para simplificar el índice.


In [4]:
# Agrupar por 'ICAO' y 'Hora'
grupos = df_vuelos.groupby(['ICAO', 'Hora'])

# Función para seleccionar un par de mensajes con CPR Even y Odd
def seleccionar_dos_mensajes(grupo):
    grupo = grupo.sort_values(by='Timestamps')

    for i in range(len(grupo) - 1):
        if grupo.iloc[i]['type_code'] in range(5, 9) and grupo.iloc[i + 1]['type_code'] in range(5, 9):

            if grupo.iloc[i]["CPR format"] != grupo.iloc[i + 1]["CPR format"]:  # Asegurar un par Even-Odd
                return grupo.iloc[[i, i + 1]]

    return pd.DataFrame()

# Aplicar la función a cada grupo
resultados = grupos.apply(seleccionar_dos_mensajes)

# Resetear el índice del DataFrame resultante
resultados = resultados.reset_index(drop=True)

# Mostrar el resultado
resultados

C:\Users\nacho\AppData\Local\Temp\ipykernel_34964\3057514726.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultados = grupos.apply(seleccionar_dos_mensajes)


,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format
0,1.733059e+12,8c0101ba3e4f362a0287a168a1ef,0101ba,False,2024-12-01 13:10:26,82.000,323.4375,0.0,13.0,7.0,Odd
1,1.733059e+12,8c0101ba3e4f33f6c07366ca2cf9,0101ba,False,2024-12-01 13:10:27,82.000,323.4375,0.0,13.0,7.0,Even
2,1.733063e+12,8c0101ba381e461b3c82ccd8ef34,0101ba,False,2024-12-01 14:29:01,0.000,281.2500,0.0,14.0,7.0,Odd
3,1.733063e+12,8c0101ba381e43e7a06e7f7dcd8f,0101ba,False,2024-12-01 14:30:15,0.000,281.2500,0.0,14.0,7.0,Even
4,1.733065e+12,8c0101ba3a8813e7f86f21fe395b,0101ba,False,2024-12-01 15:00:00,16.000,2.8125,0.0,15.0,7.0,Even
...,...,...,...,...,...,...,...,...,...,...,...
3487,1.733036e+12,8fe80475392eb62e7a8545b02c37,e80475,<NA>,2024-12-01 07:00:00,4.500,300.9375,0.0,7.0,7.0,Odd
3488,1.733055e+12,8fe80475382e16304a833cd1770d,e80475,<NA>,2024-12-01 12:09:04,0.125,272.8125,0.0,12.0,7.0,Odd
3489,1.733055e+12,8fe80475382e13fd0a6ef188b7bb,e80475,<NA>,2024-12-01 12:09:04,0.125,272.8125,0.0,12.0,7.0,Even
3490,1.733064e+12,8ce94c883ebf2622488377dacf71,e94c88,False,2024-12-01 14:35:23,96.000,320.6250,0.0,14.0,7.0,Odd


Vemos las coordenadas de un mensaje

In [5]:
msg0, msg1 = resultados['message'].iloc[2:4].values
t0, t1 = resultados['ts_kafka'].iloc[2:4].values

msg0, msg1, t0, t1

# Posición del radar (referencia)
lat_ref = 40.51  # Latitud del radar
lon_ref = -3.53  # Longitud del radar

# Obtener la posición del avión
posicion = pms.adsb.position(msg0, msg1, t0, t1, lat_ref, lon_ref)

print(posicion[0], posicion[1])  # Devuelve (latitud, longitud)

48.77724857653602 2.8795563547234906


Se define una función para calcular las posiciones geográficas a partir de pares de mensajes consecutivos, utilizando la información de los mensajes y sus marcas de tiempo (`ts_kafka`). La función calcula la latitud y longitud para cada par y agrega estas columnas al DataFrame. Luego, se aplica esta función al DataFrame `resultados` y se muestran los primeros resultados.


In [6]:
import pandas as pd

# Definimos una función para calcular la posición y agregarla al DataFrame
def calcular_posicion(grupo):
    latitudes = []
    longitudes = []
    
    # Iterar sobre cada par de mensajes consecutivos
    for i in range(0, len(grupo) - 1, 2):  # saltar de 2 en 2 (par de mensajes consecutivos)
        msg0 = grupo.iloc[i]['message']
        msg1 = grupo.iloc[i + 1]['message']
        t0 = grupo.iloc[i]['ts_kafka']
        t1 = grupo.iloc[i + 1]['ts_kafka']
        
        # Calcular la posición utilizando los dos mensajes consecutivos
        posicion = pms.adsb.position(msg0, msg1, t0, t1, lat_ref, lon_ref)
        lat = posicion[0]
        lon = posicion[1]
        
        latitudes.append(lat)
        longitudes.append(lon)
    
    # Añadir las posiciones al DataFrame
    grupo['latitud'] = latitudes * 2  # Multiplicamos por 2 porque cada latitud corresponde a dos filas consecutivas
    grupo['longitud'] = longitudes * 2  # Igual para la longitud
    
    return grupo

# Aplicar la función al DataFrame completo
resultados = calcular_posicion(resultados)

# Mostrar los primeros resultados
resultados.head()

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format,latitud,longitud
0,1.733059e+12,8c0101ba3e4f362a0287a168a1ef,0101ba,False,2024-12-01 13:10:26,82.0,323.4375,0.0,13.0,7.0,Odd,48.799780,2.902234
1,1.733059e+12,8c0101ba3e4f33f6c07366ca2cf9,0101ba,False,2024-12-01 13:10:27,82.0,323.4375,0.0,13.0,7.0,Even,48.777249,2.879556
2,1.733063e+12,8c0101ba381e461b3c82ccd8ef34,0101ba,False,2024-12-01 14:29:01,0.0,281.2500,0.0,14.0,7.0,Odd,40.464846,-3.565906
3,1.733063e+12,8c0101ba381e43e7a06e7f7dcd8f,0101ba,False,2024-12-01 14:30:15,0.0,281.2500,0.0,14.0,7.0,Even,40.476275,-3.567186
4,1.733065e+12,8c0101ba3a8813e7f86f21fe395b,0101ba,False,2024-12-01 15:00:00,16.0,2.8125,0.0,15.0,7.0,Even,48.809730,2.881634


Se eliminan las filas impares del DataFrame, conservando solo las filas de índices pares, y se asegura que la columna `Timestamps` esté en formato `datetime`. Luego, el DataFrame se ordena por la columna `Timestamps` y se resetea el índice para mantener la estructura adecuada.


In [7]:
# Eliminar filas impares (quedarse con solo las filas de índices pares)
resultados = resultados.iloc[::2].reset_index(drop=True)
# Asegurarse de que la columna 'Timestamps' esté en formato datetime
resultados['Timestamps'] = pd.to_datetime(resultados['Timestamps'])

# Ordenar el DataFrame por la columna 'Timestamps'
resultados = resultados.sort_values(by='Timestamps').reset_index(drop=True)
resultados

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format,latitud,longitud
0,1.733011e+12,8c3441152b78e1bce06544f61162,344115,False,2024-12-01 00:00:00,31.000,39.3750,0.0,0.0,5.0,Even,40.485527,-3.548287
1,1.733011e+12,9134108f402008015a6eb7838a7c,34108f,<NA>,2024-12-01 00:00:00,0.125,NaN,0.0,0.0,8.0,Even,40.466359,-3.568387
2,1.733011e+12,8c343650381003ff64689fb6fb28,343650,False,2024-12-01 00:00:00,0.000,NaN,0.0,0.0,7.0,Even,48.785733,2.875473
3,1.733011e+12,8ce8044f39af062d6481630fe847,e8044f,False,2024-12-01 00:00:01,8.500,315.0000,0.0,0.0,7.0,Odd,50.300391,2.955378
4,1.733011e+12,8c342107401e03f73068f60ced21,342107,False,2024-12-01 00:00:06,0.000,270.0000,0.0,0.0,8.0,Even,48.798837,2.903318
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1.733084e+12,8c34750f3dff33f5a873d06d55b7,34750f,False,2024-12-01 20:09:16,72.000,323.4375,0.0,20.0,7.0,Even,48.811146,2.919476
1742,1.733084e+12,8c4cadff3bdef623a682e88c5c74,4cadff,False,2024-12-01 20:09:55,37.000,312.1875,0.0,20.0,7.0,Odd,40.457328,-3.568044
1743,1.733084e+12,8c343183391933e2ea6c3dd4aa02,343183,False,2024-12-01 20:11:37,4.000,53.4375,0.0,20.0,7.0,Even,40.473947,-3.572211
1744,1.733084e+12,8f34761438100624f08127769c9f,347614,<NA>,2024-12-01 20:11:59,0.000,NaN,0.0,20.0,7.0,Odd,40.473132,-3.557666


Se carga el archivo CSV con Dask y se convierte la columna `Timestamps` a formato `datetime`, extrayendo también la hora. Luego, se calcula el `type_code` de los mensajes ADS-B y se filtran los registros correspondientes a mensajes en el rango de `type_code` [9, 18], que están relacionados con la posición en vuelo. Después, se aplica una función para determinar el formato CPR (Odd o Even) de los mensajes y se añade esta información como una nueva columna `'CPR format'`.


In [8]:
# Ruta al archivo CSV
path = Path("output_by_day/data_2024-12-01.csv")

# Leer el archivo CSV con Dask
df_vuelos = dd.read_csv(path)

# Convertir la columna Timestamps a datetime
df_vuelos["Timestamps"] = dd.to_datetime(df_vuelos["Timestamps"])

# Extraer la hora de la columna Timestamps
df_vuelos["Hora"] = df_vuelos["Timestamps"].dt.hour

df_vuelos['type_code'] = df_vuelos.map_partitions(
    lambda df: df['message'].apply(lambda msg: pms.common.typecode(msg)),
    meta=('message', 'int64') 
)

# Filtrar las filas cuyo 'type_code' esté en el rango [5, 8] o [9, 18]
df_vuelos_filtrado = df_vuelos[df_vuelos['type_code'].between(9, 18, inclusive='both')]

# Función para determinar el formato CPR (Odd o Even)
def determinar_cpr_format(row):
    tc = row['type_code']
    msg = row['message']
    if 9 <= tc <= 18:  # Surface position
        oe = adsb.oe_flag(msg)
        return "Odd" if oe else "Even"
    else:
        return None

# Aplicar la función para añadir la columna 'CPR format'
df_vuelos_filtrado['CPR format'] = df_vuelos_filtrado.map_partitions(
    lambda df: df.apply(determinar_cpr_format, axis=1),
    meta=('CPR format', 'object')
)

In [9]:
# Computar el DataFrame filtrado
df_vuelos = df_vuelos_filtrado.compute()

# Mostrar el resultado
df_vuelos

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format
11,1733011203260,8d34510a58ab05bf9a3aa61579ae,34510a,True,2024-12-01 00:00:03,NaN,NaN,NaN,0,11.0,Odd
14,1733011203260,8d4cada458c902cc68e460c3075b,4cada4,True,2024-12-01 00:00:03,NaN,NaN,NaN,0,11.0,Even
33,1733011203261,8d34640e58a562604260ec123dcd,34640e,True,2024-12-01 00:00:03,NaN,NaN,NaN,0,11.0,Even
76,1733011207114,8d02012b58bf0636414e824e7a61,02012b,True,2024-12-01 00:00:07,NaN,NaN,NaN,0,11.0,Odd
77,1733011207114,8d34640e58a595ef106956b48bde,34640e,True,2024-12-01 00:00:07,NaN,NaN,NaN,0,11.0,Odd
...,...,...,...,...,...,...,...,...,...,...,...
891748,1733084001652,8d440237582b76a1e71accfd5320,440237,True,2024-12-01 20:13:21,NaN,NaN,NaN,20,11.0,Odd
891759,1733084001652,8d4ca97b58d302bd7ae3d109b36e,4ca97b,True,2024-12-01 20:13:21,NaN,NaN,NaN,20,11.0,Even
891778,1733084001652,8d407cb058b986ab530bef82ca3b,407cb0,True,2024-12-01 20:13:21,NaN,NaN,NaN,20,11.0,Odd
891780,1733084001652,8d49514f58b9a379288ba33d7e32,49514f,True,2024-12-01 20:13:21,NaN,NaN,NaN,20,11.0,Even


Se agrupan los datos por `ICAO` y `Hora`, y se define una función para seleccionar pares de mensajes consecutivos con formatos CPR "Even" y "Odd", asegurando que estén en el orden correcto. Esta función se aplica a cada grupo de vuelos y el resultado se almacena en un nuevo DataFrame `resultados_air`, cuyo índice se resetea para mayor claridad.


In [10]:
# Agrupar por 'ICAO' y 'Hora'
grupos = df_vuelos.groupby(['ICAO', 'Hora'])

# Función para seleccionar un par de mensajes con CPR Even y Odd
def seleccionar_dos_mensajes(grupo):
    grupo = grupo.sort_values(by='Timestamps')

    for i in range(len(grupo) - 1):
        if grupo.iloc[i]['type_code'] in range(9, 19) and grupo.iloc[i + 1]['type_code'] in range(9, 19):

            if grupo.iloc[i]["CPR format"] != grupo.iloc[i + 1]["CPR format"]:  # Asegurar un par Even-Odd
                return grupo.iloc[[i, i + 1]]

    return pd.DataFrame()

# Aplicar la función a cada grupo
resultados_air = grupos.apply(seleccionar_dos_mensajes)

# Resetear el índice del DataFrame resultante
resultados_air = resultados_air.reset_index(drop=True)

# Mostrar el resultado
resultados_air

C:\Users\nacho\AppData\Local\Temp\ipykernel_34964\2729630463.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultados_air = grupos.apply(seleccionar_dos_mensajes)


,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format
0,1.733043e+12,8d01007d78d747e00000005eb8d9,01007d,True,2024-12-01 08:53:17,NaN,NaN,NaN,8.0,15.0,Odd
1,1.733043e+12,8d01007d78d7480000000031014c,01007d,True,2024-12-01 08:53:21,NaN,NaN,NaN,8.0,15.0,Even
2,1.733058e+12,8d0101ba58596674417591caf42a,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Odd
3,1.733058e+12,8d0101ba585952e6fb725ea8afae,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Even
4,1.733058e+12,8d0101ba583932db0947da0f664d,0101ba,True,2024-12-01 13:00:21,NaN,NaN,NaN,13.0,11.0,Even
...,...,...,...,...,...,...,...,...,...,...,...
8175,1.733056e+12,8de80475580d968ca3204f90bb3e,e80475,True,2024-12-01 12:24:45,NaN,NaN,NaN,12.0,11.0,Odd
8176,1.733062e+12,8de94c8858af8225b4278de99992,e94c88,True,2024-12-01 13:59:24,NaN,NaN,NaN,13.0,11.0,Even
8177,1.733062e+12,8de94c8858af85b684320694a346,e94c88,True,2024-12-01 13:59:26,NaN,NaN,NaN,13.0,11.0,Odd
8178,1.733062e+12,8de94c8858af8230122d1b3a6c91,e94c88,True,2024-12-01 14:00:05,NaN,NaN,NaN,14.0,11.0,Even


Se define una función para calcular las posiciones geográficas a partir de pares de mensajes consecutivos, utilizando los mensajes y sus marcas de tiempo (`ts_kafka`). Si la función de cálculo de posición devuelve un valor válido, se agregan las coordenadas al DataFrame. En caso de error, se asignan valores nulos. Luego, se aplica esta función al DataFrame `resultados_air` y se muestran los primeros resultados.


In [11]:
# Definimos una función para calcular la posición y agregarla al DataFrame
def calcular_posicion(grupo):
    latitudes = []
    longitudes = []
    
    # Iterar sobre cada par de mensajes consecutivos
    for i in range(0, len(grupo) - 1, 2):  # saltar de 2 en 2 (par de mensajes consecutivos)
        msg0 = grupo.iloc[i]['message']
        msg1 = grupo.iloc[i + 1]['message']
        t0 = grupo.iloc[i]['ts_kafka']
        t1 = grupo.iloc[i + 1]['ts_kafka']
        
        # Calcular la posición utilizando los dos mensajes consecutivos
        posicion = pms.adsb.position(msg0, msg1, t0, t1, lat_ref, lon_ref)
        if posicion is not None:  # Verificar que la función no devuelva None
            lat, lon = posicion
        else:
            lat, lon = None, None  # En caso de error, guardar valores nulos
        
        latitudes.append(lat)
        longitudes.append(lon)
    
    # Añadir las posiciones al DataFrame
    grupo['latitud'] = latitudes * 2  # Multiplicamos por 2 porque cada latitud corresponde a dos filas consecutivas
    grupo['longitud'] = longitudes * 2  # Igual para la longitud
    
    return grupo

# Aplicar la función al DataFrame completo
resultados_air = calcular_posicion(resultados_air)

# Mostrar los primeros resultados
resultados_air

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format,latitud,longitud
0,1.733043e+12,8d01007d78d747e00000005eb8d9,01007d,True,2024-12-01 08:53:17,NaN,NaN,NaN,8.0,15.0,Odd,12.000000,0.000000
1,1.733043e+12,8d01007d78d7480000000031014c,01007d,True,2024-12-01 08:53:21,NaN,NaN,NaN,8.0,15.0,Even,40.353378,-2.213013
2,1.733058e+12,8d0101ba58596674417591caf42a,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Odd,40.283374,-2.879042
3,1.733058e+12,8d0101ba585952e6fb725ea8afae,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Even,40.507370,-3.559204
4,1.733058e+12,8d0101ba583932db0947da0f664d,0101ba,True,2024-12-01 13:00:21,NaN,NaN,NaN,13.0,11.0,Even,41.919891,-6.437489
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8175,1.733056e+12,8de80475580d968ca3204f90bb3e,e80475,True,2024-12-01 12:24:45,NaN,NaN,NaN,12.0,11.0,Odd,39.569309,-7.848450
8176,1.733062e+12,8de94c8858af8225b4278de99992,e94c88,True,2024-12-01 13:59:24,NaN,NaN,NaN,13.0,11.0,Even,38.660339,-5.548454
8177,1.733062e+12,8de94c8858af85b684320694a346,e94c88,True,2024-12-01 13:59:26,NaN,NaN,NaN,13.0,11.0,Odd,40.499004,-3.574614
8178,1.733062e+12,8de94c8858af8230122d1b3a6c91,e94c88,True,2024-12-01 14:00:05,NaN,NaN,NaN,14.0,11.0,Even,39.223147,-7.218384


In [12]:
# Contar valores None en latitud y longitud
none_latitud = resultados_air['latitud'].isna().sum()
none_longitud = resultados_air['longitud'].isna().sum()

print(f"Valores None en latitud: {none_latitud}")
print(f"Valores None en longitud: {none_longitud}")


Valores None en latitud: 46
Valores None en longitud: 46


In [13]:
# Eliminar filas donde latitud o longitud sean None (NaN)
resultados_air = resultados_air.dropna(subset=['latitud', 'longitud']).reset_index(drop=True)
resultados_air = resultados_air.iloc[2:].reset_index(drop=True)

# Mostrar los primeros resultados después de la limpieza
resultados_air

,ts_kafka,message,ICAO,Capability,Timestamps,Speed,Track,Vertical rate,Hora,type_code,CPR format,latitud,longitud
0,1.733058e+12,8d0101ba58596674417591caf42a,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Odd,40.283374,-2.879042
1,1.733058e+12,8d0101ba585952e6fb725ea8afae,0101ba,True,2024-12-01 12:53:56,NaN,NaN,NaN,12.0,11.0,Even,40.507370,-3.559204
2,1.733058e+12,8d0101ba583932db0947da0f664d,0101ba,True,2024-12-01 13:00:21,NaN,NaN,NaN,13.0,11.0,Even,41.919891,-6.437489
3,1.733058e+12,8d0101ba58393668734bd66eb828,0101ba,True,2024-12-01 13:00:22,NaN,NaN,NaN,13.0,11.0,Odd,38.358124,-5.134823
4,1.733066e+12,8d0101ba580d968df5214642c6b2,0101ba,True,2024-12-01 15:13:56,NaN,NaN,NaN,15.0,11.0,Odd,42.346941,-0.635930
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8127,1.733056e+12,8de80475580d968ca3204f90bb3e,e80475,True,2024-12-01 12:24:45,NaN,NaN,NaN,12.0,11.0,Odd,39.569309,-7.848450
8128,1.733062e+12,8de94c8858af8225b4278de99992,e94c88,True,2024-12-01 13:59:24,NaN,NaN,NaN,13.0,11.0,Even,38.660339,-5.548454
8129,1.733062e+12,8de94c8858af85b684320694a346,e94c88,True,2024-12-01 13:59:26,NaN,NaN,NaN,13.0,11.0,Odd,40.499004,-3.574614
8130,1.733062e+12,8de94c8858af8230122d1b3a6c91,e94c88,True,2024-12-01 14:00:05,NaN,NaN,NaN,14.0,11.0,Even,39.223147,-7.218384


Se crea una aplicación Dash que visualiza los mensajes de aviones en un mapa interactivo. Los datos de los vuelos en tierra y en vuelo se almacenan en los DataFrames `df_tierra` y `df_volando`, respectivamente. Además, se define un DataFrame con la información del radar y las pistas, que incluye las ubicaciones y características visuales. La interfaz incluye un selector de hora y un mapa que se ajusta al tamaño completo de la ventana. La app permite al usuario consultar los vuelos en función de la hora seleccionada.


In [38]:
df_tierra = resultados
df_volando = resultados_air

# Datos del radar y pistas
df_info = pd.DataFrame({
    "ubicacion": ["Radar", "Pista 1 (32L/14R)", "Pista 2 (32R/14L)", "Pista 3 (36L/18R)", "Pista 4 (36R/18L)"],
    "lat": [40.51, 40.463, 40.473, 40.507, 40.507],
    "lon": [-3.53, -3.554, -3.536, -3.574, -3.559],
    "color": ["red", "yellow", "yellow", "yellow", "yellow"],  # Radar rojo, pistas amarillas
    "symbol": ["circle", "square", "square", "square", "square"]  # Radar como punto, pistas como cuadrados
})

_, nombre = str(path).split(sep = "\\")
_, nombre = nombre.split(sep = "_")
fecha, _ = nombre.split(sep = ".")
ano, mes, dia = fecha.split(sep = "-")
fecha = str(dia + " " + mes + " " + ano)

# Crear la app Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H2(f"Consulta de Mensajes de Aviones del {fecha}" , style={"text-align": "center"}),

    # Selector de hora en una sola línea con el texto
    html.Div([
        html.Label("Selecciona una hora:"),
        dcc.Dropdown(
            id="hora-dropdown",
            options=[{"label": f"{h}:00", "value": h} for h in range(24)],
            value=12,  # Valor por defecto (12:00)
            clearable=False,
            style={"margin-left": "10px", "width": "120px"}  # Reduce el margen y ajusta el ancho del dropdown
        )
    ], style={"display": "flex", "align-items": "center"}),  # Usa flexbox para alinear en la misma fila

    html.Br(),

    # Mapa
    html.Div([
        dcc.Graph(
            id="mapa-vuelos",
            style={
                "width": "100vw",  # 100% del ancho de la ventana
                "height": "100vh",  # 100% de la altura de la ventana
                "margin-left": "0px"  # Pegado al margen izquierdo
            }
        )
    ], style={"display": "flex", "justify-content": "flex-start", "height": "100vh"}),

    html.Br(),

], style={"height": "100vh", "padding": "20px"})

Se define un callback en la aplicación Dash para actualizar el mapa interactivo según la hora seleccionada. El callback filtra los datos de los aviones en tierra y en vuelo según la hora y genera un mapa con la ubicación del radar y las pistas. Se añaden trazas para los aviones en tierra (amarillos) y en vuelo (verdes), configurando el mapa con una proyección "natural earth" y personalizando la leyenda para las categorías. El mapa se actualiza automáticamente cuando el usuario selecciona una hora diferente.


In [39]:
@app.callback(
    Output("mapa-vuelos", "figure"),
    Input("hora-dropdown", "value")
)
def actualizar_mapa(hora):
    if hora is None:
        return px.scatter_geo()

    # Filtrar datos por la hora seleccionada
    df_tierra_filtrado = df_tierra[df_tierra["Hora"] == hora]
    df_volando_filtrado = df_volando[df_volando["Hora"] == hora]

    # Contar el número de aviones en tierra y en vuelo
    num_tierra = len(df_tierra_filtrado)
    num_volando = len(df_volando_filtrado)

    # Crear figura del mapa con el radar y las pistas
    fig = px.scatter_geo(
        df_info,
        lat="lat",
        lon="lon",
        title="Ubicación del radar, pistas y aviones",
        color="ubicacion",  # Usamos ubicacion para que la leyenda tenga nombres correctos
        symbol="symbol",
        hover_name="ubicacion",
        color_discrete_map={"Radar": "red", "Pistas": "yellow"}  # Mapeo de colores
    )

    # Añadir los aviones en tierra (amarillos)
    if not df_tierra_filtrado.empty:
        trace_tierra = px.scatter_geo(
            df_tierra_filtrado,
            lat="latitud",
            lon="longitud",
            hover_name="ICAO",
            hover_data={"Timestamps": True},
            color_discrete_sequence=["yellow"]
        ).data[0]
        trace_tierra.name = "Aviones en tierra"  # Nombre personalizado en la leyenda
        trace_tierra.showlegend = True
        fig.add_trace(trace_tierra)

    # Añadir los aviones en vuelo (verdes)
    if not df_volando_filtrado.empty:
        trace_volando = px.scatter_geo(
            df_volando_filtrado,
            lat="latitud",
            lon="longitud",
            hover_name="ICAO",
            hover_data={"Timestamps": True},
            color_discrete_sequence=["green"]
        ).data[0]
        trace_volando.name = "Aviones en vuelo"  # Nombre personalizado en la leyenda
        trace_volando.showlegend = True
        fig.add_trace(trace_volando)

    # Configurar el mapa
    fig.update_geos(projection_type="natural earth")
    fig.update_traces(
        marker=dict(size=10, line=dict(width=2, color="black")),
        hoverinfo="text",
    )

    # Personalizar la leyenda
    fig.update_layout(
        showlegend=True,
        legend_title="Categorías",
        legend=dict(
            itemsizing="constant",
            traceorder="normal",
            title_font=dict(size=14),
            font=dict(size=12)
        )
    )

    # Mover la anotación con el número de aviones debajo de la leyenda
    fig.add_annotation(
        x=1.25,  # Posición horizontal (0 = izquierda, 1 = derecha)
        y=0.55,  # Posición vertical ajustada para que esté debajo de la leyenda
        text=f"Aviones en tierra: {num_tierra}<br>Aviones en vuelo: {num_volando}",
        showarrow=False,
        font=dict(size=12, color="black"),
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        xref="paper",  # Usar coordenadas relativas al gráfico
        yref="paper"
    )

    return fig

In [40]:
# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server(debug=True)

C:\Users\nacho\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\dash.py:1815: DeprecationWarning:

'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead

